<a href="https://colab.research.google.com/github/goktuggokmen/Sentiment-Analysis-of-IMDB-Movie-Reviews/blob/main/Sentiment_Analysis_of_IMDB_Movie_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **<font color='red'>Problem Tanımı</font>**

**Projemizde kullanılmak üzere seçilen IMDb veri seti csv uzantılıdır,50000 (elli bin) veri içermektedir ve "reviews (incelemeler)" ve "sentiment (duygu)" olmak üzere 2 kolondan oluşmaktadır. "reviews" kolonu IMDb.com sitesindeki çeşitli filmlere yapılan değerlendirmeleri içermektedir ve "sentiment" kolonu da bu değerlendirmelerin olumlu mu olumsuz mu olduğu bilgisini içermektedir.Çeşitli classifierlar oluşturularak yorumların olumlu mu olumsuz mu olduğu tahmin edilmeye çalışılacaktır. Daha sonra oluşturulan bu modellerin performansları birbirleriyle karşılaştırılıcaktır.**  

# **<font color='red'>1. Kütüphanelerin Yüklenmesi ve Verinin Dosyadan Okunması</font>**

**<font color='red'>a. Kütüphanelerin Yüklenmesi</font>**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Kullanilacak kutuphanelerin yuklenmesi
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import os
print(os.listdir("/content/drive/MyDrive/Colab Notebooks/Machine Learning/Ders Projesi/"))
import warnings
warnings.filterwarnings('ignore')

['IMDB_Dataset.csv', 'IMDb Film İncelemelerinin Duygu Analizi.ipynb']


**<font color='red'>b. Verinin Dosyadan Okunması</font>**

In [ ]:
#Verinin dosyadan okunmasi
imdb_data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Machine Learning/Ders Projesi/IMDB_Dataset.csv')
print("Veri setinin boyutu:", imdb_data.shape)
imdb_data.head(10)

Veri setinin boyutu: (50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


**<font color='black'>Veri seti daha önce de bahsettiğimiz gibi 2 sütun ve 50000 (elli bin) satırdan oluşmaktadır.</font>**

# **<font color='red'>2. Veri Keşfi</font>**

In [ ]:
#Veri setinin ozeti
imdb_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


**<font color='black'>"review" kolonuna ait 50000 değerlendirmeden 49582 tanesi unique (eşsiz) ve 'positive' etiketli bir değerlendirme 5 kez yapılmıştır (aynı yorum 5 kez yapılmıştır).</font>**

In [ ]:
#olumlu ve olumsuz degerlendirme sayisi
imdb_data["sentiment"].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

**<font color='black'>Olumlu ve olumsuz değerlendirmelerin her ikisi de 25000 (yirmi beş bin) tane olup dengeli dağılmıştır.</font>**

# **<font color='red'>3.Text Preprocessing (Metin Ön işleme)</font>**

**<font color='black'>"review" kolonundaki değerlendirme verilerine çeşitli ön işleme işlemleri uygulayarak veriyi işlemeye uygun hale getirirken yardım alacağımız kütüphane NLTK kütüphanesi olacak. NLTK(Natural Language Toolkit) 'Doğal Dil Araç Takımı' anlamına gelir. NLTK, insan dili verileriyle çalışmak için Pyhton programlama dili ile geliştirilmiş ve geliştirilmekte olan 50’nin üzerinde derlem(corpus) ve sözcük kaynağı(lexical resources) ile oluşturulmuş açık kaynaklı bir kütüphanedir. Bu kütüphanede bir takım modüller de mevcuttur, bu modüller verilerimizi ön işlemeden geçirirken, makine öğrenmesi algoritmalarını kullanırken, Twitter API ile işlemler yaparken vs. kullanacağımız paketlerdir. Bu küpüthanenin işlevlerine örnek olarak bizim de az sonra kullanacağımız, bir cümledeki kelimeleri ayırma (Tokenization) ve kelimede var olan ekleri kaldırıp kökü bulma (Stemming) örnek verilebilir.</font>**

**<font color='red'>a.Normalizasyon</font>**

In [ ]:
#NLTK kutuphanesinin import edilmesi ve 'stopwords'lerin indirilmesi
import nltk
nltk.download('stopwords')

#Metin tokenizasyonu
tokenizer=ToktokTokenizer()

#Ingilizce stopwordlerin ayarlanmasi
stopword_list=nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**<font color='red'>b.Gürültü Kaldırma</font>**

In [ ]:
#HTML seritlerini kaldiran fonksiyon tanimlama
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Koseli parantezleri kaldiran fonksiyon tanimlama
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Gurultuleri kaldiran fonksiyon tanimlama
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

#"review" kolonuna gurultu kaldirma fonksiyonunun uygulanmasi
imdb_data['review']=imdb_data['review'].apply(denoise_text)

**<font color='red'>c.Özel Karakterleri Kaldırma</font>**

**<font color='black'>Veri setimizde İngilizce değerlendirmelerle çalıştığımız için özel karakterlerin silindiğinden emin olmamız gerekiyor.</font>**

In [ ]:
#Ozel karakterleri kaldiran fonksiyon tanimlama
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

#"review" kolonuna fonksiyonun uygulanmasi
imdb_data['review']=imdb_data['review'].apply(remove_special_characters)

**<font color='red'>d.Stemming</font>**

**<font color='black'>Stemming (köklendirme), ek almış kelimeleri aldığı eklerden arındırma veya İngilizcede zaman (tense) nedeniyle değişime uğramış kelimelerin kökünü tespit etme işlemidir.</font>**

In [ ]:
#Stemming fonksiyonu tanimlama
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text

#"review" kolonuna fonksiyonun uygulanmasi
imdb_data['review']=imdb_data['review'].apply(simple_stemmer)

**<font color='red'>e.Stopwords (Etkisiz Kelimeler) Kaldırma</font>**

**<font color='black'>Durdurma sözcükleri, çok az anlamı olan veya hiç anlamı olmayan sözcüklerdir.</font>**

In [ ]:
stop=set(stopwords.words('english'))
print(stop)

#stopwordleri kaldiran fonksiyonu tanimlama
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

#"review" kolonuna fonksiyonun uygulanmasi
imdb_data['review']=imdb_data['review'].apply(remove_stopwords) 

{'each', 'i', 'theirs', "wouldn't", 'ourselves', 'he', 'very', 'no', 'when', 'you', "hadn't", 'but', 'his', 'herself', 'than', 'she', 'o', 'my', 'itself', 'yours', "shouldn't", 'through', 'doing', 'if', 'not', 'y', "couldn't", 'ma', "didn't", 'has', 'once', 'should', 'all', 'shan', 'wouldn', 'how', "mightn't", 'couldn', 'we', 'yourselves', 'll', 'are', 'both', "needn't", 'doesn', 'or', 'by', 'such', 'having', 'then', 'd', 'it', "it's", 'won', 'was', 'an', 'our', 'had', 'off', 'a', "don't", 're', 'with', 'into', 'needn', 'those', 'him', 'does', "you'll", 'why', 'out', 'between', 's', 'been', 'have', 'weren', 'any', 'for', 'myself', 'before', 'against', 'being', 've', 'only', 'on', "hasn't", 'under', "you've", 'they', 'hasn', 'did', 'mustn', 'were', "mustn't", 'which', "should've", 'about', 'here', 'its', 'other', 'will', 'until', 'am', 'do', 'ain', 'mightn', 'further', 'ours', 'again', 'more', 'and', "isn't", 'this', "aren't", 'the', 'these', 'few', 'just', 'them', "you'd", "that'll", '

# **<font color='red'>4.Veri Kümesini Train ve Test Verisi Olarak Ayırma</font>**

In [ ]:
#train dataset
train_reviews=imdb_data.review[:40000]
train_sentiments=imdb_data.sentiment[:40000]
#test dataset
test_reviews=imdb_data.review[40000:]
test_sentiments=imdb_data.sentiment[40000:]
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)

(40000,) (40000,)
(10000,) (10000,)


# **<font color='red'>5.Bag of Words Modeli (Kelime Çantası Modeli) Oluşturma</font>**

**<font color='black'>Bag Of Words (Kelime Çantası): Metin belgelerini sayısal vektörlere veya kelime torbasına dönüştürmek için kullanılır.</font>**

**<font color='black'>Sklearn kitaplığından önceden oluşturulmuş CountVectorizer fonksiyonunu kullanacağız.</font>**


In [ ]:
#Bag of Words icin countVectorizer olusturma
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))

#train datasini donusturme
cv_train_reviews=cv.fit_transform(train_reviews)

#test datasini donusturme
cv_test_reviews=cv.transform(test_reviews)

print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)

BOW_cv_train: (40000, 6209089)
BOW_cv_test: (10000, 6209089)


# **<font color='red'>6.TF-IDF (Term Frequency-Inverse Document Frequency) Modeli Oluşturma</font>**

**<font color='black'>Metin belgelerini TF-IDF özellik matrisine dönüştürmek için kullanılır.</font>**

**<font color='black'>Term Frequency-Inverse Document Frequency (Terim Frekansı-Ters Belge Frekansı), bir kelimenin bir belge için ne kadar önemli olduğunun sayısal bir ölçüsüdür.</font>**

In [ ]:
#TF-IDF vectorizer 
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))

#train datasini donusturme
tv_train_reviews=tv.fit_transform(train_reviews)

#test datasini donusturme
tv_test_reviews=tv.transform(test_reviews)

print('TF-IDF_train:',tv_train_reviews.shape)
print('TF-IDF_test:',tv_test_reviews.shape)

TF-IDF_train: (40000, 6209089)
TF-IDF_test: (10000, 6209089)


**<font color='black'>Kelime Vektörleştirme olarak da bilinen Kelime Gömme, bir sözlükten karşılık gelen bir gerçek sayı vektörüne kelimeleri veya cümleleri eşlemek için kullanılan bir NLP (Natural Language Processing)  tekniğidir. Vektörleştirme, kelimeleri sayılara çevirme işlemidir.</font>**

# **<font color='red'>7.Duygu Verisini Binary Etiketlere Dönüştürme</font>**

**<font color='red'>a.Etiketleme</font>**

**<font color='black'>Label Binarizer, Kategorik verileri alan ve bir Numpy dizisi çıkaran bir SciKit Learn sınıfıdır.</font>**

In [ ]:
#"sentiment" verisini etiketleme
lb=LabelBinarizer()

#donusturulmus "sentiment" verisi
sentiment_data=lb.fit_transform(imdb_data['sentiment'])

print(sentiment_data.shape)

(50000, 1)


**<font color='red'>b.Train ve Test Olarak Ayırma</font>**

In [ ]:
#Duygu verilerini bolme
train_sentiments=sentiment_data[:40000]
test_sentiments=sentiment_data[40000:]
print(train_sentiments)
print(test_sentiments)

[[1]
 [1]
 [1]
 ...
 [1]
 [0]
 [0]]
[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


# **<font color='red'>8.Modelleri Oluşturma ve Performanslarını Değerlendirme</font>**

# **<font color='red'>a.Logistic Regression (Lojistik Regresyon)</font>**

**<font color='red'>a.1.Modelin Oluşturulması ve Bag of Words ve TF-IDF İçin Uydurulması</font>**

**<font color='black'>Hem Bag of Words hem de TF-IDF özellikleri için lojistik regresyon modeli oluşturacağız.</font>**


In [ ]:
#Modeli egitme
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)

#Modeli Bag of Words icin uydurma
lr_bow=lr.fit(cv_train_reviews,train_sentiments)
print(lr_bow)

#Modeli TF-IDF icin uydurma
lr_tfidf=lr.fit(tv_train_reviews,train_sentiments)
print(lr_tfidf)

LogisticRegression(C=1, max_iter=500, random_state=42)
LogisticRegression(C=1, max_iter=500, random_state=42)


**<font color='red'>a.2.Test Veri Seti Üzerinde Lojistik Regresyon Modeli Performansı</font>**

In [ ]:
#Bag of Words kullanan modelin tahminleri
lr_bow_predict=lr.predict(cv_test_reviews)
print(lr_bow_predict)

#TF-IDF kullanan modelin tahminleri
lr_tfidf_predict=lr.predict(tv_test_reviews)
print(lr_tfidf_predict)

[0 0 0 ... 0 1 1]
[0 0 0 ... 0 1 1]


**<font color='red'>a.3.Modelin Doğruluğu (Accuracy)</font>**

In [ ]:
#Bag of Words icin accuracy
lr_bow_score=accuracy_score(test_sentiments,lr_bow_predict)
print("lr_bow_score :",lr_bow_score)

#TF-IDF icin accuracy
lr_tfidf_score=accuracy_score(test_sentiments,lr_tfidf_predict)
print("lr_tfidf_score :",lr_tfidf_score)

lr_bow_score : 0.7512
lr_tfidf_score : 0.75


**<font color='red'>a.4.Classification Report (Sınıflandırma Raporu)</font>**

In [ ]:
#Bag of Words ozelligi icin classification raporu
lr_bow_report=classification_report(test_sentiments,lr_bow_predict,target_names=['Positive','Negative'])
print("Bag of Words classification report:",'\n', lr_bow_report)

#TF-IDF ozelligi icin classification raporu
lr_tfidf_report=classification_report(test_sentiments,lr_tfidf_predict,target_names=['Positive','Negative'])
print("TF-IDF classification report:",'\n', lr_tfidf_report)

Bag of Words classification report: 
               precision    recall  f1-score   support

    Positive       0.75      0.75      0.75      4993
    Negative       0.75      0.75      0.75      5007

    accuracy                           0.75     10000
   macro avg       0.75      0.75      0.75     10000
weighted avg       0.75      0.75      0.75     10000

TF-IDF classification report: 
               precision    recall  f1-score   support

    Positive       0.74      0.77      0.75      4993
    Negative       0.76      0.73      0.75      5007

    accuracy                           0.75     10000
   macro avg       0.75      0.75      0.75     10000
weighted avg       0.75      0.75      0.75     10000



**<font color='red'>a.5.Confusion Matrix (Karışıklık Matrisi)</font>**

**<font color='black'>Karışıklık matrisi, bir sınıflandırma modelinin (veya sınıflandırıcının) gerçek değerleri bilinen bir dizi test verisi üzerinde ne kadar iyi performans gösterdiğini gösteren bir tablodur.</font>**

In [ ]:
#Bag of Words icin confusion matrix
cm_bow=confusion_matrix(test_sentiments,lr_bow_predict,labels=[1,0])
print(cm_bow)
#TF-IDF icin confusion matrix
cm_tfidf=confusion_matrix(test_sentiments,lr_tfidf_predict,labels=[1,0])
print(cm_tfidf)

[[3768 1239]
 [1249 3744]]
[[3663 1344]
 [1156 3837]]


# **<font color='red'>b.Linear Support Vector Machine (Doğrusal Destek Vektör Makinesi)</font>**

**<font color='red'>b.1.Modelin Oluşturulması ve Bag of Words ve TF-IDF İçin Uydurulması</font>**

**<font color='black'>Hem Bag of Words hem de TF-IDF özellikleri için Doğrusal SVM modeli oluşturacağız.</font>**

In [ ]:
#Modeli egitme
svm=SGDClassifier(loss='hinge',max_iter=500,random_state=42)

#Modeli Bag of Words icin uydurma
svm_bow=svm.fit(cv_train_reviews,train_sentiments)
print(svm_bow)

#Modeli TF-IDF icin uydurma
svm_tfidf=svm.fit(tv_train_reviews,train_sentiments)
print(svm_tfidf)

SGDClassifier(max_iter=500, random_state=42)
SGDClassifier(max_iter=500, random_state=42)


**<font color='red'>b.2.Test Veri Seti Üzerinde Lojistik Regresyon Modeli Performansı</font>**

In [ ]:
#Bag of Words kullanan modelin tahminleri
svm_bow_predict=svm.predict(cv_test_reviews)
print(svm_bow_predict)

#TF-IDF kullanan modelin tahminleri
svm_tfidf_predict=svm.predict(tv_test_reviews)
print(svm_tfidf_predict)

[1 1 0 ... 1 1 1]
[1 1 1 ... 1 1 1]


**<font color='red'>b.3.Modelin Doğruluğu (Accuracy)</font>**

In [ ]:
#Bag of Words icin accuracy
svm_bow_score=accuracy_score(test_sentiments,svm_bow_predict)
print("svm_bow_score :",svm_bow_score)

#TF-IDF icin accuracy
svm_tfidf_score=accuracy_score(test_sentiments,svm_tfidf_predict)
print("svm_tfidf_score :",svm_tfidf_score)

svm_bow_score : 0.5829
svm_tfidf_score : 0.5112


**<font color='red'>b.4.Classification Report (Sınıflandırma Raporu)</font>**

In [ ]:
#Bag of Words ozelligi icin classification raporu
svm_bow_report=classification_report(test_sentiments,svm_bow_predict,target_names=['Positive','Negative'])
print("Bag of Words classification report:",'\n',svm_bow_report)

#TF-IDF ozelligi icin classification raporu
svm_tfidf_report=classification_report(test_sentiments,svm_tfidf_predict,target_names=['Positive','Negative'])
print("TF-IDF classification report:",'\n',svm_tfidf_report)

Bag of Words classification report: 
               precision    recall  f1-score   support

    Positive       0.94      0.18      0.30      4993
    Negative       0.55      0.99      0.70      5007

    accuracy                           0.58     10000
   macro avg       0.74      0.58      0.50     10000
weighted avg       0.74      0.58      0.50     10000

TF-IDF classification report: 
               precision    recall  f1-score   support

    Positive       1.00      0.02      0.04      4993
    Negative       0.51      1.00      0.67      5007

    accuracy                           0.51     10000
   macro avg       0.75      0.51      0.36     10000
weighted avg       0.75      0.51      0.36     10000



**<font color='red'>b.5.Confusion Matrix (Karışıklık Matrisi)</font>**

In [ ]:
#Bag of Words icin confusion matrix
cm_bow=confusion_matrix(test_sentiments,svm_bow_predict,labels=[1,0])
print(cm_bow)

#TF-IDF icin confusion matrix
cm_tfidf=confusion_matrix(test_sentiments,svm_tfidf_predict,labels=[1,0])
print(cm_tfidf)

[[4948   59]
 [4112  881]]
[[5007    0]
 [4888  105]]
